In [1]:
# Importamos los módulos
import pandas as pd
import numpy as np
from pyhrv import time_domain as td
from pyhrv import frequency_domain as frd

In [2]:
# Leemos el diario con la actividad y la existencia de estrés
diario = pd.read_csv('diario.csv', sep = ';')

In [3]:
# Leemos los valores de la frecuencia cardiaca
df = pd.read_csv('HRV_Extractor.csv')

In [4]:
# Calculamos el tiempo entre intervalos RR (en ms) a partir de la frecuencia cardiaca con la ecuación 1.1
df['RR (ms)'] = 60000/df['bpm']

In [5]:
df

,bpm,time,set,creationDate,measureDate,RR (ms)
0,92,"19:02:42,97",0,2021-04-06 19:07:44 +0200,"2021-04-06 19:02:42,97",652.173913
1,94,"19:02:43,61",0,2021-04-06 19:07:44 +0200,"2021-04-06 19:02:43,61",638.297872
2,96,"19:02:44,23",0,2021-04-06 19:07:44 +0200,"2021-04-06 19:02:44,23",625.000000
3,93,"19:02:44,88",0,2021-04-06 19:07:44 +0200,"2021-04-06 19:02:44,88",645.161290
4,92,"19:02:45,53",0,2021-04-06 19:07:44 +0200,"2021-04-06 19:02:45,53",652.173913
...,...,...,...,...,...,...
29984,84,"17:11:13,27",141,2021-05-12 17:11:51 +0200,"2021-05-12 17:11:13,27",714.285714
29985,90,"17:11:13,94",141,2021-05-12 17:11:51 +0200,"2021-05-12 17:11:13,94",666.666667
29986,86,"17:11:14,64",141,2021-05-12 17:11:51 +0200,"2021-05-12 17:11:14,64",697.674419
29987,80,"17:11:38,62",141,2021-05-12 17:11:51 +0200,"2021-05-12 17:11:38,62",750.000000


In [6]:
# Creamos las funciones para calcular la media, desviación típica, máximos, mínimos, RMSSD, NN50 y pNN50
def mean(df):
    return np.mean(df)
def SD(df):
    return np.std(df)
def max_(df):
    return np.max(df)
def min_(df):
    return np.min(df)
def RMSSD(df):
    return np.sqrt(np.mean(np.square(np.diff(df))))
def NN50(df):
    return np.sum(np.abs(np.diff(df)) > 50)*1
def pNN50(df):
    return 100 * np.sum((np.abs(np.diff(df)) > 50)*1) / len(df)

In [7]:
# Aplicamos las funciones previamente definidas al dataframe tanto a la frecuencia cardiaca como a la variabilidad de la FC
td = df.groupby(['set'])['RR (ms)'].apply(mean).to_frame(name = 'Mean RR (ms)').reset_index()
td['count'] = df.groupby(['set'])['RR (ms)'].count()
td['SDNN'] = df.groupby(['set'])['RR (ms)'].apply(SD)
td['Max RR (ms)'] = df.groupby(['set'])['RR (ms)'].apply(max_)
td['Min RR (ms)'] = df.groupby(['set'])['RR (ms)'].apply(min_)
td['Mean HR (bpm)'] = df.groupby(['set'])['bpm'].apply(mean)
td['SDHR'] = df.groupby(['set'])['bpm'].apply(SD)
td['Max HR (bpm)'] = df.groupby(['set'])['bpm'].apply(max_)
td['Min HR (bpm)'] = df.groupby(['set'])['bpm'].apply(min_)
td['RMSSD (ms)'] = df.groupby(['set'])['RR (ms)'].apply(RMSSD)
td['NN50'] = df.groupby(['set'])['RR (ms)'].apply(NN50)
td['pNN50 (%)'] = df.groupby(['set'])['RR (ms)'].apply(pNN50)

In [8]:
td.head()

,set,Mean RR (ms),count,SDNN,Max RR (ms),Min RR (ms),Mean HR (bpm),SDHR,Max HR (bpm),Min HR (bpm),RMSSD (ms),NN50,pNN50 (%)
0,0,666.732651,400,36.011889,810.810811,576.923077,90.252500,4.856824,104,74,24.516799,18,4.500000
1,1,675.845275,358,43.625605,800.000000,576.923077,89.139665,5.628890,104,75,26.564615,22,6.145251
2,2,706.045426,367,50.528900,882.352941,566.037736,85.414169,6.095750,106,68,30.957175,34,9.264305
3,3,719.336600,403,49.793026,909.090909,588.235294,83.821340,5.967475,102,66,31.012516,35,8.684864
4,4,730.020615,404,55.302622,895.522388,600.000000,82.660891,6.250652,100,67,36.224291,52,12.871287


In [9]:
%%capture
# Calculamos los valores en el dominio de la frecuencia mediante el método de Welch 
freq = df.groupby(['set'])['RR (ms)'].apply(frd.welch_psd)

In [10]:
# Extraemos los valores que nos interesan (potencia absoluta)
freq.to_csv("freq.csv", header=None, index=False)

In [11]:
freq = pd.read_csv('freq.csv', sep=', ', header=None)

<ipython-input-11-c603d9dd2208>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  freq = pd.read_csv('freq.csv', sep=', ', header=None)


In [12]:
freq[10]

0      fft_abs=(208.33521944522502
1      fft_abs=(226.01483337596548
2        fft_abs=(986.242824199413
3       fft_abs=(782.6171042008428
4        fft_abs=(994.073274267952
                  ...             
137    fft_abs=(1038.0228786435716
138     fft_abs=(302.7461075815251
139    fft_abs=(444.56659905650105
140    fft_abs=(178.64190064586356
141     fft_abs=(1719.211859572644
Name: 10, Length: 142, dtype: object

In [13]:
VLF = freq[10].str.split("(", n=1, expand=True)
HF = freq[12].str.split(")", n=1, expand=True)
LF_HF = freq[21].str.split("=", n=1, expand=True)

In [14]:
# Creamos un dataframe con los datos que nos interesan
fd = pd.DataFrame()
fd['VLF'] = VLF[1]
fd['LF'] = freq[11]
fd['HF'] = HF[0]
fd['LF/HF'] = LF_HF[1]

In [15]:
fd.head()

,VLF,LF,HF,LF/HF
0,208.33521944522502,876.063350,359.35023720659444,2.4379094813096853
1,226.01483337596548,1061.738081,284.7311017304538,3.728915015871623
2,986.242824199413,1194.980991,750.3700955449714,1.5925221405642385
3,782.6171042008428,1312.166983,431.49047294806974,3.041010324768007
4,994.073274267952,1624.758069,542.2965814884626,2.9960691700771136


In [16]:
# Unimos en un dataframe los valores en el dominio del tiempo y de la frecuencia y el diario
df2 = pd.concat([td, fd, diario], axis=1)

In [17]:
df2

,set,Mean RR (ms),count,SDNN,Max RR (ms),Min RR (ms),Mean HR (bpm),SDHR,Max HR (bpm),Min HR (bpm),RMSSD (ms),NN50,pNN50 (%),VLF,LF,HF,LF/HF,Actividad,Stress
0,0,666.732651,400,36.011889,810.810811,576.923077,90.252500,4.856824,104,74,24.516799,18,4.500000,208.33521944522502,876.063350,359.35023720659444,2.4379094813096853,Estudio Tarde,0
1,1,675.845275,358,43.625605,800.000000,576.923077,89.139665,5.628890,104,75,26.564615,22,6.145251,226.01483337596548,1061.738081,284.7311017304538,3.728915015871623,Estudio Tarde,0
2,2,706.045426,367,50.528900,882.352941,566.037736,85.414169,6.095750,106,68,30.957175,34,9.264305,986.242824199413,1194.980991,750.3700955449714,1.5925221405642385,Estudio Tarde,0
3,3,719.336600,403,49.793026,909.090909,588.235294,83.821340,5.967475,102,66,31.012516,35,8.684864,782.6171042008428,1312.166983,431.49047294806974,3.041010324768007,Estudio Mañana,0
4,4,730.020615,404,55.302622,895.522388,600.000000,82.660891,6.250652,100,67,36.224291,52,12.871287,994.073274267952,1624.758069,542.2965814884626,2.9960691700771136,Estudio Mañana,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,137,676.025979,160,56.473084,821.917808,560.747664,89.368750,7.403227,107,73,46.060465,29,18.125000,1038.0228786435716,1991.400335,874.390943506942,2.2774713643243794,Previo Examen 4,1
138,138,675.121755,135,32.798522,759.493671,588.235294,89.081481,4.303404,102,79,30.279710,15,11.111111,302.7461075815251,164.981020,341.67912590618585,0.4828536707433977,Examen 4,1
139,139,657.339803,103,37.899371,750.000000,576.923077,91.582524,5.312605,104,80,41.655710,15,14.563107,444.56659905650105,276.743086,240.60790496289195,1.1501828525943123,Examen 4,1
140,140,679.033020,187,46.439402,810.810811,582.524272,88.770053,6.005840,103,74,31.408721,19,10.160428,178.64190064586356,540.772617,489.93767747386005,1.1037579717336492,Examen 4,1


In [18]:
# Exportamos el dataframe a un archivo csv
df2.to_csv("HRV_Analysis.csv", index=False)